## 4. Create Azure ML Solution
In this section, we will create an inference engine wrapper, a class that will get an image data as input, analyse it and return analysis result as a json formatted data. We will be using Intel OpenVino runtime for analysis.

### 4.1. Get global variables
We will read the previously stored variables. We need the name of the directory that we will use to store in our ml solution files. We create a directory with the specified directory name (if not already exist)

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

isSolutionPath = get_key(envPath, "isSolutionPath")

import os
if not os.path.exists(isSolutionPath):
    os.mkdir(isSolutionPath)

### 4.2. Download OpenVino ML model
Download Intel's OpenVino Intermediate Representation models (BIN + XML files) for object detections. We will download multiple models where each have different capabilities. For more details about the capabilities of these models please refer to the links provided below.  

ModelZoo can be found at: https://github.com/opencv/open_model_zoo/blob/master/models/intel/index.md  
Please click the model names in the ModelZoo page to see model and its performance details.  

As reference sample, we will be downloading following models and make them ready to be used in the sample:
```
# person detectors
    "person-detection-retail-0002", 
    "person-detection-retail-0013", 
    "pedestrian-detection-adas-0002", 
    "pedestrian-detection-adas-binary-0001", 
# person + vehicle detectors
    "pedestrian-and-vehicle-detector-adas-0001",
    "person-vehicle-bike-detection-crossroad-0078",
    "person-vehicle-bike-detection-crossroad-1016",
# vehicle detectors
    "vehicle-detection-adas-0002",
    "vehicle-detection-adas-binary-0001",
    "vehicle-license-plate-detection-barrier-0106"
```

Lets download these models and store under the ML Solition directory.

In [ ]:
import os
import yaml
import urllib.request


def getModel(modelName, isSolutionPath):
    mlSolutionModelFilePath = os.path.join(".", isSolutionPath, "models", modelName)

    if not os.path.exists(mlSolutionModelFilePath):
        url = "https://raw.githubusercontent.com/opencv/open_model_zoo/master/models/intel/{0}/model.yml".format(modelName)
        resp = urllib.request.urlopen(url)
        yamlFileContent = resp.read()

        yamlContent = yaml.load_all(yamlFileContent)
        for c in yamlContent:
            for k1, v1 in c.items():
                if k1 == "files":
                    for k2, v2 in enumerate(v1):
                        for k in v2:
                            if k == "name":
                                modelPathName = v2[k] 
                            if k == "source":
                                modelSourceURL = v2[k]
                                # Download the model
                                pathDownload = os.path.join(mlSolutionModelFilePath, modelPathName)
                                headTail = os.path.split(pathDownload) 
                                os.makedirs(headTail[0], exist_ok=True)
                                res = urllib.request.urlretrieve(modelSourceURL, pathDownload)
                                print("{} downloaded".format(pathDownload))
                                break
                    break
    else:
        print("{} already exists here, so not downloading again.".format(mlSolutionModelFilePath))


# see full list of models at Model Zoo...
modelNames = [  # person detectors
                "person-detection-retail-0002", 
                "person-detection-retail-0013", 
                "pedestrian-detection-adas-0002", 
                "pedestrian-detection-adas-binary-0001", 
                # person + vehicle detectors
                "pedestrian-and-vehicle-detector-adas-0001",
                "person-vehicle-bike-detection-crossroad-0078",
                "person-vehicle-bike-detection-crossroad-1016",
                # vehicle detectors
                "vehicle-detection-adas-0002",
                "vehicle-detection-adas-binary-0001",
                "vehicle-license-plate-detection-barrier-0106"]

for modelName in modelNames:
    print("Downloading: {0}".format(modelName))
    getModel(modelName, isSolutionPath)

### 4.3. Create Inferenec Engine Wrapper
HEre we create a class that will have different properties and methods to help scoring, analysing an image data. This class will also help us to specify analytics compute target such as CPU, VPU, FPGA etc. and also debugging features.

In [ ]:
%%writefile $isSolutionPath/score.py
# Copyright [yyyy] [name of copyright owner]

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import cv2
import json
import logging
import numpy as np
import timeit as t
from openvino.inference_engine import IENetwork, IEPlugin
from collections import OrderedDict
import threading

logging.basicConfig(level=logging.DEBUG)

class AnalyticsAPI:
    INF_STAT_OK = 0             # OK
    INF_STAT_NOT_READY = 1      # Scoring engine is not ready
    INF_STAT_EXCEPTION = 2      # Exception occured whiled inferencing

    # targetDev: "CPU", "MYRIAD", "GPU", "FPGA"
    def __init__(self, modelName, modelPrecision, targetDev, probThreshold, pluginPath=None, cpuExtensions=None):
        try:
            self.initialized = False

            self.logger = logging.getLogger("AnalyticsAPILogger")
            self.modelPath = "./models"

            self.modelName = modelName
            self.modelPrecision = modelPrecision
            self.targetDev = targetDev
            self.probThreshold = probThreshold
            self.pluginPath = pluginPath
            self.cpuExtensions = cpuExtensions
            self._lock = threading.Lock()

            self.initEngine()

        except Exception as e:
            self.logger.info("[AI EXT] Exception (AnalyticsAPI/__init__): {0}".format(str(e)))
            return None

    def initEngine(self):
        try:
            with self._lock:
                self.initialized = False
                self.logger.info("[AI EXT] AnalyticsAPI init start.")
                start = t.default_timer()

                self.modelXMLFileName = os.path.join(self.modelPath, self.modelName, self.modelPrecision, self.modelName + ".xml")
                self.modelBINFileName = os.path.join(self.modelPath, self.modelName, self.modelPrecision, self.modelName + ".bin")

                # Init compute target...
                self.iePlugin = IEPlugin(device=self.targetDev, plugin_dirs=self.pluginPath)
                if self.cpuExtensions and 'CPU' in self.targetDev:
                    self.iePlugin.add_cpu_extension(self.cpuExtensions)

                ieNet = IENetwork(model=self.modelXMLFileName, weights=self.modelBINFileName)

                assert len(ieNet.inputs.keys()) == 1, "Only single input topologies supported!"
                assert len(ieNet.outputs) == 1, "Only single output topologies supported!"
                self.inputBlob = next(iter(ieNet.inputs))
                self.outBlob = next(iter(ieNet.outputs))

                self.ieExecNet = self.iePlugin.load(network=ieNet, num_requests=2)

                # Read and pre-process input image
                self.ieNetShape = ieNet.inputs[self.inputBlob].shape  # n, c, h, w

                self.initialized = True

                end = t.default_timer()
                self.logger.info("[AI EXT] AnalyticsAPI init end. Duration: {0} ms".format(round((end - start) * 1000, 2)))

        except Exception as e:
            self.logger.info("[AI EXT] Exception (ScoringAPI/initModel): {0}".format(str(e)))
            return None

    def setProbabilityThreshold(self, probThreshold=0.5):
        self.probThreshold = probThreshold
        self.logger.info("[AI EXT] (setProbabilityThreshold): {0}".format(probThreshold))

    def getProbabilityThreshold(self):
        return self.probThreshold

    def setModel(self, modelName):
        self.modelName = modelName
        self.initEngine()
        self.logger.info("[AI EXT] (setModelName): {0}".format(modelName))

    def getModelName(self):
        return self.modelName

    def setTargetDevice(self, targetDev):
        self.targetDev = targetDev
        self.initEngine()
        self.logger.info("[AI EXT] (setTargetDevice): {0}".format(targetDev))

    def getTargetDevice(self):
        return self.targetDev

    def setModelPrecision(self, modelPrecision):
        self.modelPrecision = modelPrecision
        self.initEngine()
        self.logger.info("[AI EXT] (setModelPrecision): {0}".format(modelPrecision))

    def getModelPrecision(self):
        return self.modelPrecision

    def preprocess(self, cvImage):
        ih, iw = cvImage.shape[:-1]
        imageHW = (ih, iw)

        if (ih, iw) != (self.ieNetShape[2], self.ieNetShape[3]):
            cvImage = cv2.resize(cvImage, (self.ieNetShape[3], self.ieNetShape[2]))

        cvImage = cvImage.transpose((2, 0, 1))  # Change data layout from HWC to CHW

        cvImage = cvImage.reshape(self.ieNetShape)

        return cvImage, imageHW

    def postprocess(self, infRes, imageHW):
        resDict = OrderedDict()

        objectId = 0
        for obj in infRes[self.outBlob][0][0]:
            if obj[2] > self.probThreshold:
                xmin = int(obj[3] * imageHW[1])
                ymin = int(obj[4] * imageHW[0])
                xmax = int(obj[5] * imageHW[1])
                ymax = int(obj[6] * imageHW[0])
                classId = int(obj[1])

                resDict[objectId] = {   "label": classId, 
                                        "confidence": round(float(obj[2]), 2), 
                                        "xmin": xmin, 
                                        "ymin": ymin, 
                                        "xmax": xmax, 
                                        "ymax": ymax}
                objectId += 1

        return resDict

    def score(self, cvImage):
        try:
            #self.logger.info("[AI EXT] Scoring start: {0}".format(self.initialized))
            with self._lock:
                if self.initialized:
                    image, imageHW = self.preprocess(cvImage)

                    start = t.default_timer()
                    infRes = self.ieExecNet.infer(inputs={self.inputBlob: image})
                    end = t.default_timer()
                    infTime = round((end - start) * 1000, 4)
                    
                    resDict = self.postprocess(infRes, imageHW)

                    result = {  "status": self.INF_STAT_OK,
                                "time" : infTime,
                                "object_count" : len(resDict),
                                "result": resDict}

                    result = json.dumps(result)

                else:
                    resJson = OrderedDict()
                    resJson[0] = {"status": self.INF_STAT_NOT_READY}
                    result = json.dumps(resJson)

            #self.logger.info("[AI EXT] Scoring end: {0}".format(self.initialized))
            return result

        except Exception as e:
            self.logger.info("[AI EXT] Exception (ScoringAPI - Score): {0}".format(str(e)))
            resJson = OrderedDict()
            resJson[0] = {"status": self.INF_STAT_EXCEPTION}
            result = json.dumps(resJson)
            return result

    def version(self):
        return "v 1.0"

    def about(self):
        aboutString = "Engine initialized: {0}<br>ModelName: {1}<br>ModelPrecision: {2}<br>TargetDev: {3}<br>ProbThreshold: {4}<br>PluginPath: {5}<br>CpuExtensions: {6}".format(self.initialized, self.modelName, self.modelPrecision, self.targetDev, self.probThreshold, self.pluginPath, self.cpuExtensions)
        return aboutString

Score method of the above InferenceEngine class will return the result as Json formated string.  

Result will be one of the below three Json string:  

1) {"status": 1}  
Above result means the Inference Engine is not ready for inferencing...

2) {"status": 2}  
Above result means an exception occured while scoring. Details are in the logs...

3) Third option is below where status equals to 0. So before you you process the result, you can check the status to see if it contains valid detection data, status code.

Fields below are self descriptive and you can refere to above source code for details.


```
{
   "status": 0,
   "time": 45.6475,
   "object_count": 100,
   "result": {
      "0": {
         "label": 1,
         "confidence": 1,
         "xmin": 304,
         "ymin": 169,
         "xmax": 398,
         "ymax": 436
      },
      "1": {
         "label": 2,
         "confidence": 0.99,
         "xmin": 474,
         "ymin": 201,
         "xmax": 586,
         "ymax": 485
      },

        ...

      "100": {
         "label": 5,
         "confidence": 0.53,
         "xmin": 385,
         "ymin": 140,
         "xmax": 421,
         "ymax": 246
      }
   }
}
```